<a href="https://colab.research.google.com/github/shimseonjo/DDV/blob/main/%EC%BD%94%EB%9E%A9%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('hi')

hi


In [1]:
%%time

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as f
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

CPU times: user 5.08 s, sys: 646 ms, total: 5.73 s
Wall time: 8.9 s


In [2]:
%%time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cifar10_train =  CIFAR10('./data',train=True,download=True,transform=transforms.ToTensor())
cifar10_test = CIFAR10('./data',train=False,download=True,transform=transforms.ToTensor())

batch_size = 64
train_loader = DataLoader(cifar10_train,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(cifar10_test,batch_size=len(cifar10_test),shuffle=False)


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,256)
        self.fc2 = nn.Linear(256,10)

    def forward(self,x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

recode_loss_train=[]
recode_loss_test=[]

x_test,y_test = next(iter(test_loader))
x_test,y_test = x_test.to(device),y_test.to(device)

for i in range(10):
  net.train()
  loss_train = 0.0
  for j,(x_train,y_train) in enumerate(train_loader):
    x_train,y_train = x_train.to(device),y_train.to(device)
    y = net(x_train)
    loss = loss_func(y,y_train)
    loss_train += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  loss_train /= (j+1)
  recode_loss_train.append(loss_train)

  net.eval()
  y_test_pred = net(x_test)
  loss_test = loss_func(y_test_pred,y_test)
  recode_loss_test.append(loss_test.item())

# CPU times: user 4min 32s, sys: 8.62 s, total: 4min 41s
# Wall time: 4min 47s

# CPU times: user 1min 24s, sys: 1.82 s, total: 1min 25s
# Wall time: 1min 31s

cuda:0


100%|██████████| 170M/170M [00:03<00:00, 46.5MB/s]


CPU times: user 1min 24s, sys: 1.82 s, total: 1min 25s
Wall time: 1min 31s
